In [25]:
import pandas as pd
import os
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import scipy.sparse
from scipy.sparse import csr_matrix

# Create sparse matrix from given BoW

In [22]:
nrow = 0
data = []
row = []
col= []
#with open('data401/reviews/train/labeldBow.feat') as pearl:
with open('data401/reviews/test/labeledBow.feat') as pearl:
    text = pearl.read().split('\n')
    for line in text:
        if len(line) > 0:
            entries = line.split(" ")
            row.append(nrow)
            col.append(0)
            if int(entries[0]) > 5: 
                data.append(1)
            else:
                data.append(0)
            c= 1
            while c < len(entries):
                ent = entries[c].split(":")
                row.append(nrow)
                col.append(int(ent[0]) + 1)
                data.append(int(ent[1]))
                c+=1
            nrow += 1
sp = csr_matrix((data, (row, col)))

In [24]:
#scipy.sparse.save_npz('origfeat_train.npz',sp)
scipy.sparse.save_npz('origfeat_test.npz', sp)

# Parse files and create basic dataframe

In [8]:
reviews = []
pos = []
for filename in os.listdir('data401/reviews/train/neg/'):
    path = os.path.join('data401/reviews/train/neg/', filename)
    if not os.path.isdir(path):
        with open(os.path.join('data401/reviews/train/neg/', filename)) as file:
            data = file.read().replace("<br />","")
            reviews.append(data)
            pos.append(0)
print("halfway!")
for filename in os.listdir('data401/reviews/train/pos/'):
    path = os.path.join('data401/reviews/train/pos/', filename)
    if not os.path.isdir(path):
        with open(os.path.join('data401/reviews/train/pos/', filename)) as file:
            data = file.read().replace("<br />","")
            reviews.append(data)
            pos.append(1)
dat = {'review':reviews, 'pos': pos}
df = pd.DataFrame(data=dat)

halfway!


In [7]:
df.to_csv('reviews.csv',encoding = "utf8",index=False)

In [14]:
df['pos'].to_csv('classes.csv',encoding='utf-8',index=False)

# Unigrams and bigrams with TFIDF

In [2]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(PorterStemmer().stem(item))
    return stems

In [17]:
tfidf = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1,2), max_features=500000)
tfs = tfidf.fit_transform(df['review'])

In [19]:
scipy.sparse.save_npz('bigrams2.npz', tfs)

In [12]:
#ignore this cell!
token_dict = {}
for dirpath, dirs, files in os.walk('data401/reviews/train/neg/'):
    for f in files:
        fname = os.path.join(dirpath, f)
        with open(fname) as pearl:
            text = pearl.read()
            token_dict[f] = text.lower().translate(string.punctuation)

for dirpath, dirs, files in os.walk('data401/reviews/train/pos/'):
    for f in files:
        fname = os.path.join(dirpath, f)
        with open(fname) as pearl:
            text = pearl.read()
            token_dict[f] = text.lower().translate(string.punctuation)

halfway!
